**From the inspect element, we see that the data, which we are interested in,  
is stored in the table, wikitable, and we can access it through the table,   
tr, and td tags. Let us begin by importing and installing the necessary libraries.**

In [66]:
# import requests for getting the HTML contents, lxml.html for parsing, and pandas 
!pip install lxml
import requests
import lxml.html as lh
import pandas as pd

**Send an HTTP request to the URL of the webpage we wish to get access to,  
and the server responds by by returning the HTML content of the webpage**

In [67]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Create a handle to handle the contents of the website
tor = requests.get(URL) 

#Store the contents of the website under doc
tor_M = lh.fromstring(tor.content)

#Parse the data that is stored in the rows of the table, in tr tag 
tor_M_tr = tor_M.xpath('//tr')

In [68]:
#Check the length of the first 12 rows
[len(T) for T in tor_M_tr[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [70]:
# let’s parse the first row as our header.
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tor_M_tr[0]:
    i+=1
    name=t.text_content()
    name=name.strip() # remove the \n that follows each name
    print ("%d: %s" %(i,name))
    col.append((name,[]))
    
col # look at the tuple called col to see if everything is the way it should be


1: Postal code
2: Borough
3: Neighborhood


[('Postal code', []), ('Borough', []), ('Neighborhood', [])]

**Now we begin to read data from the webpage and store the results row by row.**

In [71]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tor_M_tr)):
    #T is our j'th row
    T=tor_M_tr[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

**Create the pandas dataframe**

In [73]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df.replace('\n', '',regex=True) #replace all \n from the entries in the dataframe
df.head() #look at the first five rows


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [75]:
# Drop the rows that have Not assigned as a Borough entry, and reset the index.
df.drop(df.loc[df['Borough'] == "Not assigned"].index, inplace=True)
df=df.reset_index()
del df['index']

# Look at the last five rows of the table.
df.tail()


,Postal code,Borough,Neighborhood
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...
102,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
103,,Canadian postal codes,


In [76]:
# The last row of df should be deleted
df_tor=df.loc[:102,:] # form a new dataframe which excludes the last row of df.
df_tor.shape

(103, 3)

**Now we will get geospatial data from a csv file, and add the longitude and   
latitude values of the neighborhoods, forming a single dataframe**

In [78]:
# Sort the dataframe df_tor by the Postal code column and set the index
df_sorted=df_tor.sort_values('Postal code')
df_sorted=df_sorted.reset_index()
del df_sorted['index']
df_sorted


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [80]:
# read the latitude and longitude values into a data frame
geo_tor=pd.read_csv("http://cocl.us/Geospatial_data")
geo_tor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [81]:
# delete the Postal code column from geospatial dataframe
del geo_tor['Postal Code'] 
geo_tor


,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476
...,...,...
98,43.706876,-79.518188
99,43.696319,-79.532242
100,43.688905,-79.554724
101,43.739416,-79.588437


In [82]:
# Create a single data frame from the two dataframes
df_sorted = pd.concat([df_sorted, geo_tor], axis=1)
df_sorted

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
